Ref
https://colab.research.google.com/github/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb#scrollTo=SaPAR7TWmxoM

# Load data

In [1]:
import wandb
run = wandb.init()
artifact = run.use_artifact('dylanon/text-summarization-model/slices:v2', type='dataset')
artifact_dir = artifact.download()

print(artifact_dir)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 63050123. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact slices:v2, 6632.04MB. 11 files... 
wandb:   11 of 11 files downloaded.  
Done. 0:0:8.6


.\artifacts\slices-v2


In [2]:
# show head of data in slice_i.parquet
import pandas as pd
df = pd.read_parquet(artifact_dir + '/slice_8.parquet')
display(df)

,id,text,sum
0,7,"spanish property group colonial , struggling u...",spain is colonial posts #.## billion euro loss
1,15,south korea is inflation rate slowed slightly...,skorea is inflation rate slows slightly in au...
2,23,south korean shares fell sharply late monday m...,skorean stocks sharply lower in morning session
3,29,hundreds of australian holidaymakers stranded ...,charter flight to bring home australians stran...
4,34,malaysia is financial markets are closed on m...,malaysia markets closed for holiday
...,...,...,...
916615,9149011,The song charted in June when Sheeran is secon...,Ed Sheeran is track Thinking Out Loud has topp...
916616,9149017,Welsh ministers are hoping to use EU money to ...,The Welsh Government has been urged by a UK go...
916617,9149019,"The incident took place in Thorpe St Andrew, N...",A man has been arrested on suspicion of murder...
916618,9149028,Alderney coastal defences were attacked by hug...,"A severe storm with conditions described as ""h..."


# Load pretrain model

In [3]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

c:\Users\nattanon\.platformio\penv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\nattanon\.platformio\penv\Lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this m

##### example - use midel

In [4]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = """การแข่งขันตะกร้อ ซูเปอร์ ซีรี่ส์ ที่ประเทศสิงคโปร์ รอบชิงชนะเลิศ ทีมชาติไทยผ่านเข้าชิงชนะเลิศทั้งประเภทชายและหญิง โดยทีมชายไทยที่ผ่านอินเดียมาได้ในรอบรองพบกับอินโดนีเซีย ที่เอาชนะเกาหลีใต้มาได้ แมตช์นี้ไทยยังรักษาสถิติไม่เคยเสียเซตแรกให้กับทีมใดก่อน เมื่อเป็นฝ่ายชนะอินโดนีเซีย 15-9 คะแนน ส่วนเซตที่ 2 ทีมไทยชนะ อินโดนีเซีย 15-8 และเซตที่ 3 ทีมไทยชนะ 15-3 ทำให้ทีมไทย ชนะอินโดนีเซีย 3-0 เซต คว้าแชมป์ไปครองส่วนทีมหญิงไทย ดีกรีอันดับ 1 ของโลก รอบชิงชนะเลิศพบกับอันดับที่สองของโลกอย่าง เวียดนาม ซึ่งปรากฏว่าสาวไทยเอาชนะเวียดนามไป 3 เซตรวด 15-9 15-9 และ 15-13 คว้าแชมป์ไปครอง สำหรับตะกร้ออิสแทฟ ซูเปอร์ ซีรี่ส์ รายการต่อไปจะมาแข่งขันที่กรุงเทพ ในเดือนมิถุนายน"""

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]
print(input_ids)
output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]


summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

tensor([[   259,  87168,  69260,  58384, 140980,    259,  85461, 180411, 165600,
         117548,  29373,  17636,  47414, 188445, 160082, 130621, 223122,    259,
         182965,  11984,  23043,  17138, 130621, 223122,  18281,  76027,  53959,
           2091,  85350,   9521,  35835,  53959,  11984,   1549,  23043, 168764,
         222168, 228309,  90982, 146348,  76389,  67864,  95320, 170504,  17636,
         210805,  82967,  89051, 222168,    259,  98179,   6196, 162969,   4682,
          11984,  21746,  34904, 151489, 109663,  37087,  45204,   6196,  43427,
          46865,  35835,  44954,  22087,  21942,   3682,  66806,  66936,  76389,
          67864,  95320, 170504,  28032,    495,  84679,  51738,  45204,   6196,
           1549,    356, 159695,  11984,  66936,    259,  76389,  67864,  95320,
         170504,    670,  11817,   3324,  45204,   6196,   1549,    381, 159695,
          11984,  66936,    670,   4935,  58094,  35835,  11984,    259,  66936,
          76389,  67864,  95

# Process data

In [14]:
# Process data
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

def preprocess_data(dataset):
  # Convert the text and summary columns to tensors.
  input_text = dataset["text"]
  target_text = dataset["sum"]

  # this line just follow model card in huggingface
  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
  input_text = [WHITESPACE_HANDLER(text) for text in input_text]

  # Tokenize the input and target text.
  input_ids = tokenizer.batch_encode_plus( input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)["input_ids"]
  target_ids = tokenizer.batch_encode_plus( target_text, return_tensors="pt", padding=True, truncation=True, max_length=150)["input_ids"]

  return {
        "input_ids": input_ids,
        "target_ids": target_ids
  }

c:\Users\nattanon\.platformio\penv\Lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [16]:
# Prepare the data for all slices.
import datasets
import pandas as pd

# Load the data from the artifact.
df = pd.read_parquet(artifact_dir + '/slice_0.parquet')[:20000]
dataset = datasets.Dataset.from_pandas(df)

# Preprocess the data.
dataset = dataset.map(preprocess_data, batched=True)


















  0%|          | 0/10000 [04:48<?, ?it/s]




Map: 100%|██████████| 20000/20000 [00:03<00:00, 5053.21 examples/s]


In [8]:
# split data to train test and validation
def split_dataset(dataset, train_ratio=0.8, val_ratio=0.1):
  """Split a dataset into train, validation, and test splits.

  Args:
    dataset: huggingface Dataset object.
    train_ratio: The fraction of the data to use for training.
    val_ratio: The fraction of the data to use for validation.

  Returns:
    A tuple of three lists, the train, validation, and test splits.
  """
  
  # dataset = dataset.shuffle()
  
  split_dataset = dataset.train_test_split(train_size=train_ratio)
  train_set = split_dataset["train"]
  data_subset = split_dataset["test"]
  split_dataset = data_subset.train_test_split(train_size=val_ratio / (1 - train_ratio))
  val_set = split_dataset["train"]
  test_set = split_dataset["test"]

  return train_set, val_set, test_set

# Split the processed dataset into train, validation, and test splits
train_set, val_set, test_set = split_dataset(dataset)
# sacve data to local
train_set.save_to_disk("train_set")
val_set.save_to_disk("val_set")
test_set.save_to_disk("test_set")


Saving the dataset (1/1 shards): 100%|██████████| 2000/2000 [00:00<00:00, 45721.71 examples/s]


In [11]:
print(train_set)

Dataset({
    features: ['id', 'text', 'sum', 'input_ids', 'target_ids'],
    num_rows: 16000
})


In [17]:
from transformers import TrainingArguments, Trainer


# Define the training arguments with tensorflow.
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=1000,
    save_total_limit=3,
    logging_steps=1000,
    
    report_to="wandb",
    run_name="test-first-run",
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=dataset,
)

trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds